# Neuron Network Model*

In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os
import matplotlib as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import matplotlib.pyplot as plt
import time
import datetime
import Model_1Linear_1Softmax as model
import plot_confusion_matrix

---
## 0 .Data Preparation*
---

In [2]:
%%time
with open("../../Data/Feature_Engineering/df_features_normalize.p", "rb" ) as f:
    data_features = pickle.load(f)    
with open( "../../Data/Feature_Engineering/df_label.p", "rb" ) as f:
    data_label = pickle.load(f)

CPU times: user 1 s, sys: 6.59 s, total: 7.59 s
Wall time: 40.3 s


In [3]:
%%time
print('shape of data_features', data_features.shape)
print('shape of data_label', data_label.shape)
print('---------------------------------------')
X = data_features
Y0 = data_label
Y0 = np.asarray(Y0)
Y1=1-Y0
Y2=Y0
Y1=Y1.reshape(Y1.shape[0],1)
Y2=Y2.reshape(Y2.shape[0],1)
Y=np.concatenate((Y1,Y2),axis=1)

bar_70 = np.floor(X.shape[0]*0.7).astype(int)
bar_70=bar_70+1
bar_80 = np.floor(X.shape[0]*0.8).astype(int)
bar_80=bar_80+1

X_train=X[:bar_70]
print('X_train done.')

Y_train=Y[:bar_70]
Y_train=Y_train.reshape([Y_train.shape[0],Y_train.shape[1]])
print('Y_train done.')

X_validation=X[bar_70:bar_80]
print('X_validation done.')

Y_validation=Y[bar_70:bar_80]
Y_validation=Y_validation.reshape([Y_validation.shape[0],Y_validation.shape[1]])
print('Y_validation done.')

X_test=X[bar_80:]
print('X_test done.')

Y_test=Y[bar_80:]
Y_test=Y_test.reshape([Y_test.shape[0],Y_test.shape[1]])
print('Y_test done.')

print('---------------------------------------')
print('shape of X_train',X_train.shape)
print('shape of Y_train',Y_train.shape)
print('shape of X_validataion',X_validation.shape)
print('shape of Y_validation',Y_validation.shape)
print('shape of X_test',X_test.shape)
print('shape of Y_test',Y_test.shape)

shape of data_features (220047, 2357)
shape of data_label (220047,)
---------------------------------------
X_train done.
Y_train done.
X_validation done.
Y_validation done.
X_test done.
Y_test done.
---------------------------------------
shape of X_train (154033, 2357)
shape of Y_train (154033, 2)
shape of X_validataion (22005, 2357)
shape of Y_validation (22005, 2)
shape of X_test (44009, 2357)
shape of Y_test (44009, 2)
CPU times: user 3 ms, sys: 6 ms, total: 9 ms
Wall time: 45.3 ms


In [4]:
(nrTrainSamples, dimX) = X_train.shape
(nrValidationSamples, dimY)  = Y_validation.shape

print ("Finished: data loaded. Stats below: ")
print ("Number of training samples: %d" %nrTrainSamples)
print ("Number of validation  samples: %d" %nrValidationSamples)
print ("Dimension of X_train: %d" %dimX)
print ("Dimension of Y_validation: %d" %dimY)

Finished: data loaded. Stats below: 
Number of training samples: 154033
Number of validation  samples: 22005
Dimension of X_train: 2357
Dimension of Y_validation: 2


---
## 1. Parameter Setting
---

In [4]:
sess = tf.InteractiveSession()
BATCH_SIZE    = 100
MAX_ITERS     = 30
LEARNING_RATE = 0.1
require_improvement = 50  # Stop optimization if no improvement found in this many iterations.
n_hidden=256           # 256 units in this hidden lapyer

## Train Model

In [5]:
# --------------------------------------------------------------------------------------------------------------------

def trainModel(accuracy, train_op, x, y, X_train, Y_train, X_validation, Y_validation, X_test, Y_test):
#     Ensure we update the global variables rather than local copies.
#     global best_validation_accuracy
#     global last_improvement

    best_validation_accuracy=0
     
    print ("===============================")
    print ("Training started........")

    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    for indexIter in range(MAX_ITERS):
        for startIndex, endIndex in zip( range(0,len(X_train),BATCH_SIZE), range(BATCH_SIZE,len(X_train),BATCH_SIZE)):
            # gradient descending, finding parameters.
            sess.run(train_op, feed_dict = {x: X_train[startIndex:endIndex], y: Y_train[startIndex:endIndex]})
            
        # Visualize accuracy only every 10 iterations
        if (indexIter%10==0) or (indexIter == (MAX_ITERS - 1)):
            acc_train = sess.run(accuracy, feed_dict={x:X_train, y:Y_train})
            acc_validation  = sess.run(accuracy, feed_dict={x:X_validation,  y:Y_validation})
            acc_test = sess.run(accuracy, feed_dict={x:X_test,  y:Y_test})
            
            if acc_validation > best_validation_accuracy :
                # Update the best-known validation accuracy.
                best_validation_accuracy = acc_validation
                
                # Set the iteration for the last improvement to current.
                last_improvement = indexIter

                # A string to be printed below, shows improvement found.
                improved_str = '*'
                print('Iteration %d: Accuracy %.5f(training) %.5f(validation) %.5f(test)'\
                      %(indexIter+1, acc_train, acc_validation, acc_test)+improved_str)
                
                # Save all variables of the TensorFlow graph to file.
                saver = tf.train.Saver()
                save_dir='../../Data/Propensity_Modeling/Neuron_Net/model01'
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                saver.save(sess, save_dir)
                print("Model saved in file: %s" % save_dir)
                
            else:
                # An empty string to be printed below.
                # Shows that no improvement was found.
                improved_str = ''
                print('Iteration %d: Accuracy %.5f(training) %.5f(validation) %.5f(test)'\
                      %(indexIter+1, acc_train, acc_validation, acc_test)+improved_str)                

        if (indexIter - last_improvement) > require_improvement:
            print("No improvement found in a while, early stop the training.")
            print ("===============================")
            break
                  
    print ("Training finished.")
    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_diff = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(datetime.timedelta(seconds=int(round(time_diff)))))
    print ("===============================")
    



## 2. Main function

In [7]:
# -------------------------------------------------------------------------------------------------------------------
def main():
    
    #============================
    # 1. Build model (graph)
    #============================
    import tensorflow as tf
    
    x = tf.placeholder(tf.float32, shape=[None, dimX]) 
    # None is the number of data set, 784 is the dimensions of features
    y = tf.placeholder(tf.float32, shape=[None, dimY])
    
# -------------------------------------------------------------------------------------------------------------------    
    # model returns a score vector over all classes
    [logits,y_pred_prob] = model.Model(x, dimX, dimY, n_hidden)
# -------------------------------------------------------------------------------------------------------------------
    # Get predicted classes
    y_pred = tf.round(y_pred_prob)
    
    # Loss function (Cross-entropy for classification)
    cross_entropy=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits = logits))
    # Error: Only call `sigmoid_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)
    
    # Accuracy(if different measure from loss)
    correct_prediction = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #============================
    # 2. Train model
    #============================
    # initialize variables
    tf.global_variables_initializer().run()
    
    # Best validation accuracy seen so far.
    best_validation_accuracy= 0
    # Iteration-number for last improvement to validation accuracy.
    last_improvement = 0

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

    # Train model:
    train_op = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
    trainModel(accuracy, train_op, x, y, X_train, Y_train, X_validation, Y_validation, X_test, Y_test)

    acc_test = sess.run(accuracy, feed_dict={x:X_test, y:Y_test})
    y_pred_prob=sess.run(y_pred_prob, feed_dict={x:X_test})
    y_pred=sess.run(y_pred, feed_dict={x:X_test})
    print('Final model (may note be the optimal) on test data got accuracy:', acc_test)
    
    return [y_pred_prob, y_pred]

---
## 3. Trained Result:
---

### 3.1 Model: 1 Linear + 1 Sigmoid

In [19]:
#(REPRODUCIBILITY) set random seeds
tf.set_random_seed(722)

print('Model: 1 linear layer + 1 sigmoid layer')
[y_pred_prob, y_pred]=main()

Training started........
Iteration 1: Accuracy 0.88190(training) 0.88730(validation) 0.87730(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 11: Accuracy 0.88270(training) 0.88816(validation) 0.87832(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 21: Accuracy 0.88312(training) 0.88898(validation) 0.88012(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 30: Accuracy 0.88271(training) 0.88880(validation) 0.87816(test)
Training finished.
Time usage: 0:21:12
Final model (may note be the optimal) on test data got accuracy: 0.879456


In [20]:
y_pred

[<tf.Tensor 'Sigmoid:0' shape=(?, 1) dtype=float32>, array([[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 0.],
        [ 0.],
        [ 0.]], dtype=float32)]

In [18]:
sum(Y_train==0)/len(Y_train)

array([ 0.84684451])

### 3.2 Model: 1 linear + 1 softmax

In [8]:
#(REPRODUCIBILITY) set random seeds
tf.set_random_seed(722)

print('Model: 1 linear layer + 1 softmax layer')
[y_pred_prob, y_pred]=main()

Model: 1 linear layer + 1 softmax layer
Training started........
Iteration 1: Accuracy 0.88138(training) 0.88657(validation) 0.87689(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 11: Accuracy 0.88239(training) 0.88789(validation) 0.87823(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 21: Accuracy 0.88287(training) 0.88903(validation) 0.87980(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 30: Accuracy 0.88240(training) 0.88839(validation) 0.87787(test)
Training finished.
Time usage: 0:32:13
Final model (may note be the optimal) on test data got accuracy: 0.879434


In [9]:
y_pred

[<tf.Tensor 'Softmax:0' shape=(?, 2) dtype=float32>, array([[ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        ..., 
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]], dtype=float32)]

In [34]:
y_pred[1][:100,1]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

In [41]:
sum(y_pred[1][:,1]!=1&y_pred[1][:,1]!=1)

2998

In [47]:
y_pred[1].shape

(44009, 2)

In [43]:
y_pred[0].eval()

InvalidArgumentError: Shape [-1,2357] has negative dimensions
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,2357], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/ioloop.py", line 887, in start
   

### 3.2 Confusion Matrix

In [ ]:
# Compute confusion matrix
y_Test = tf.argmax(testY, 1)
y_test=y_Test.eval()
cfm = confusion_matrix(y_test, y_pred,labels=range(0,10))
np.set_printoptions(precision=2)

print('\nPlot of Confusion Matrix without nomalize is:')
plt.figure(figsize=(14,10))
plot_confusion_matrix(cfm, classes=range(0,10), normalize=False)
plt.show()
print('\nPlot of Confusion Matrix with nomalize is:')
plt.figure(figsize=(14,10))
plot_confusion_matrix(cfm, classes=range(0,10), normalize=True)
plt.show()

### End
---